# An annotation tool for image pairs

The goal is to generate pairs of images which will be annotated either as "same"=1 or "different"=0.

### Use cases:
Face recognition: training a siamese network to recognize if the same person shows up in two images\
Character recognition\
or here... are the cats in the photos the same or different?

### Input:
Some sample images from ~~work~~ my web-cam.

### Output:
Save the annotation in a parquet file with three columns: base_im, pair_im, label.

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import random
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

In [ ]:
IMAGES_PATH = Path("images")
path_list = [str(pth) for pth in IMAGES_PATH.glob("*.jpg")]
path_list[0:2]

In [ ]:
# 1. First, define the data structure for storing the results

annotations_dict = {
    "base_im":[],
    "pair_im":[],
    "label":[]
    }

In [ ]:
# 2. Define the way in which the image pairs are generated (randomly sample two paths from the list)

def generate_img_pair(input: list, k: int=2) -> tuple:
    # sample k image paths without replacement
    samples = random.sample(input, k)
    base_img_path = samples[0]
    pair_img_path = samples[1]

    assert base_img_path != pair_img_path

    return (base_img_path, pair_img_path)

In [ ]:
generate_btn = widgets.Button(description = "GENERATE")
same_btn = widgets.Button(description = "Same")
different_btn = widgets.Button(description = "Different")

In [ ]:
# At this point, the buttons are visible, but they carry no functionality or logic behind them. 
# Clicking on "GENERATE" should display a pair of images the user can assess as either "same" or "different". Let's implement this.

In [ ]:
display_plot = widgets.Output()

def generate_eventHandler(p):
    generate_btn.image_paths = generate_img_pair(input = path_list)
    images = (Image.open(generate_btn.image_paths[0]), Image.open(generate_btn.image_paths[1]))

    with display_plot:
        clear_output()
        fig, ax = plt.subplots(ncols = 2, figsize = (10, 8))
        ax[0].imshow(images[0])
        ax[1].imshow(images[1])
        display(fig.figure)
generate_btn.on_click(generate_eventHandler)

In [ ]:
display(generate_btn)
# Add the widget to display the output of the GENERATE button.
display(display_plot)

display(same_btn)
display(different_btn)

Ta-daaaa! There are the image pairs, ready to be annotated.

The next step is to add the logic to the annotation buttons. Clicking on any of the buttons should label the image pair and append the label, together with the image names, to the results dictionary. This will then overwrite the parquet file where we store the annotations.

In [ ]:
def same_eventHandler(s):
    annotations_dict["base_im"].append(generate_btn.image_paths[0])
    annotations_dict["pair_im"].append(generate_btn.image_paths[1])
    annotations_dict["label"].append(1)

def different_eventHandler(s):
    annotations_dict["base_im"].append(generate_btn.image_paths[0])
    annotations_dict["pair_im"].append(generate_btn.image_paths[1])
    annotations_dict["label"].append(0)

same_btn.on_click(same_eventHandler)
different_btn.on_click(different_eventHandler)

In [ ]:
display(generate_btn)

# Add the widget to display the output of the GENERATE button.
display(display_plot)

display(same_btn)
display(different_btn)

In [ ]:
# Check the annotations in the results dictionary
ann_df = pd.DataFrame(annotations_dict)
ann_df
# Save it to a parquet file
ann_df.to_parquet("annotations.parquet.gzip")

# Read the parquet file
# df = pd.read_parquet("annotations.parquet.gzip")